1. We load the libraries we'll need to solve the problem
2. We load the dataset that has been modified and ampliated before
3. We used a modification of LENET NN
4. We achieve 95.8% in validation set. 

In [2]:
# Load pickled data
import pickle
import numpy as np
import matplotlib as mpl
import cv2 
from   skimage  import exposure 
#TODO: Fill this in based on where you saved the training and testing data
from IPython.display import clear_output


training_file = 'train.p'
validation_file='valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

not_charged, y_valid = valid['features'], valid['labels']
not_charged, y_test = test['features'], test['labels']

# Charge augmentated and 

X_train = np.load('X_train_after_processing.dat.npy')
y_train =  np.load('Y_train_after_processing.dat.npy')
X_valid = np.load('X_valid_after_processing.dat.npy')
X_test = np.load('X_test_after_processing.dat.npy')

In [3]:
#Shuffle the data a bit... Yes, I know this is too much, but one has his maniacs behaviours...

from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train)

In [4]:
# Well use a 128 batch size. 
# We use a 30 epoch to train the net. We didn't implemented an early stop when the algorithm gets stabilized. 

import tensorflow as tf

EPOCHS = 30
BATCH_SIZE = 128

We have implemented a modified Lenet. 

Input - 32x32x1 Y channel image
Convolution 5x5 - 1x1 stride, valid padding, outputs 28x28x14
RELU
Max pooling - 2x2 stride, outputs 14x14x14
Convolution 5x5 - 1x1 stride, valid padding, outputs 10x10x16
RELU
Convolution 5x5 - 1x1 stride, valid padding, outputs 5x5x16
Fully connected - Input 400. Output 120. 
RELU
Fully connected - Input 120 - Output 84 
RELU
Dropout - Keep prob = 0.5
Fully connected - Last layer Input 84 output 43

Main differences are: 
* Layers are different in size
* Dropout has been added

In [5]:
from tensorflow.contrib.layers import flatten

def LeNet_modified(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 14), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(14))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)
   
    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
   # conv1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 14, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)
   
    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #conv2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
   
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    #fc1   = tf.nn.dropout(fc1, keep_prob) #Added dropout
    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, keep_prob) #Added dropout

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
  #  loss = tf.nn.l2_loss(fc3_W)+tf.nn.l2_loss(fc2_W)+tf.nn.l2_loss(fc1_W)
    
    return logits#,loss

In [6]:
x = tf.placeholder(tf.float32, (None, 32, 32,1))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
rate = tf.placeholder(dtype=tf.float32)
one_hot_y = tf.one_hot(y, 43)

In [7]:
#rate = 0.001
beta = 0.001


#logits,loss = LeNet(x)
logits = LeNet_modified(x)
prediction = tf.nn.softmax(logits) # for new images
top_k_prediction = tf.nn.top_k(prediction, 5) # For new images
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels=one_hot_y)#+beta*loss
#cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits= logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In evaluate function we perform operations for training and analising predictions. 

In [8]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()


def evaluate(X_data, y_data,prob):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y , keep_prob: prob})
        loss     = sess.run(loss_operation , feed_dict={x: batch_x, y: batch_y , keep_prob: prob})
        predictions   = sess.run(prediction,feed_dict={x: batch_x, y: batch_y , keep_prob: prob} )
        top_k_predictions = sess.run(top_k_prediction,feed_dict={x: batch_x, y: batch_y , keep_prob: prob})
        total_accuracy += (accuracy * len(batch_x))
    return (total_accuracy / num_examples), loss,predictions,top_k_predictions

In [ ]:
valid_vector = np.zeros(EPOCHS)
testing_accuracy_vector = np.zeros(EPOCHS)
valid_loss_vector =np.zeros(EPOCHS)
testing_loss_vector =np.zeros(EPOCHS)
counter = 0
initial_rate = 0.001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y , keep_prob: 0.5,rate: initial_rate})
            
            
        testing_accuracy,testing_loss,_,_   = evaluate(X_train,y_train, prob = 1)    
        validation_accuracy,validation_loss,_,_ = evaluate(X_valid, y_valid,prob = 1)
        valid_vector[i] = validation_accuracy
        valid_loss_vector[i] = validation_loss
        testing_accuracy_vector[i]     = testing_accuracy
        testing_loss_vector[i] = testing_loss              
                   
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Testing Accuracy = {:.3f}".format(testing_accuracy))
        print()
        
        if valid_vector[i] < valid_vector[i-1]:
            counter = counter + 1
            if counter == 2:
                counter = 0
                initial_rate = initial_rate/10.0
                print("Initial rate has been reduced {:.3f} ...".format(initial_rate))  
            
    saver.save(sess, './lenet')
    print("Model saved")

Training...



In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')

    test_accuracy,_,_,_ = evaluate(X_test, y_test, prob = 1)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    


In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')

    test_accuracy,_,_,_  = evaluate(X_valid, y_valid, prob = 1)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')

    test_accuracy,_,_,_ = evaluate(X_train, y_train, prob = 1)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
plt.plot((valid_loss_vector))
